# <center>Collaborative Filtering for Implicit Feedback Datasets

This paper is written by *Yifan Hu* in 2008. In this paper, *Hu* raise the importance of implicit data and become the first to build **implicit recommendation algorithm**. He said:

>The vast majority of the literature in the field is focused on processing explicit feedback; probably thanks to the convenience of using this kind of pure information. **However, in many practical situations recommender systems need to be centered on implicit feedback. ** This may reflect reluctance of users to rate products, or limitations of the system that is unable to collect explicit feedback.


For explicit feedback datasets,those values would be **ratings** that indicate the preference by user u of item i, where high values mean stronger preference.But for implicit feedback datasets, those values would indicate observations for user **actions.** For example, it can be the number of times u purchased item i or the time u spent on webpage i.

So instead we focus on what we know the user **has consumed** and the **confidence** we have in whether or not they like any given item. We can for example measure how often they play a song and assume a higher confidence if they’ve listened to it 500 times vs. one time. For thoese purpose *Hu* set up two variables $p_{ui}$ and $c_{ui}$.

for m users and n items:

$r_{ui}:$ the data records of user behavior.

$p_{ui}:$ a binary variables which indicates the preference of user $u$ to item $i$.

\begin{align}
p_{ui} =
\begin{cases} 
1,  & \mbox{if }r_{ui}\mbox{ > 0} \\
0, & \mbox{if }r_{ui}\mbox{ = 0}
\end{cases}\tag{1}
\end{align}

$c_{ui}:$ which measure the confidence in observing $p_{ui}$  ( $\alpha$ is learning rate, can be changed)

<center>$c_{ui}=1+\alpha r_{ui}\tag{2}$ 

Our goal is to **find a vector $x_{u} \in R^{f}$ for each user $u$, and a vector $y_{i} \in R^{f}$ for each item $i$** that will factor user preferences. In other words, preferences are assumed to be the inner products: 

<center>$p_{ui} = x^{T}_{u} y_{i}\tag{3}$ 

These vectors will be known as the user-factors and the item-factors, respectively. Essentially, the vectors strive to map users and items into a common latent factor space where they can be directly compared. This is **similar to matrix factorization techniques** which are popular for explicit feedback data, with two important **distinctions**: 

(1) We need to account for the varying **confidence levels**

(2) Optimization should **account for all possible $u$, $i$ pairs**, rather than only those corresponding to observed data. 

Accordingly, factors are computed by minimizing the following cost function:

<center>$C(x_{u},y_{i})=\sum\limits_{(u,i)\in \kappa}c_{ui}(p_{ui}-x_{u}^{T}y_{i})^{2}+\lambda(\sum\limits_{u\in \kappa}\Arrowvert x_{u}\Arrowvert^{2}+\sum\limits_{i\in \kappa}\Arrowvert y_{i}\Arrowvert^{2})\tag{4}$

Alternating Least Squares (ALS) is a the model we’ll use to fit our data and find variables. ALS is an iterative optimization process where we for every iteration try to arrive closer and closer to a factorized representation of our original data.

**Let us assume that all item-factors are gathered within an $n × f$ matrix $Y$ .  For each user u, let us define the diagonal $n × n$ matrix $C^{u}$ where $C^{u}_{ii}=c_{ui}$. And also the vector $p_{u} \in R^{n}$ that contains all the preferences by $u$ (the $p_{ui}$ values). The process of ALS is:**

process 1 : Randomly generate $Y$, and fixed

process 2 : Find partial derivation on $x_{u}$ of loss function, and get the values of $x_{u}$

\begin{align}
\frac{\partial C(x_{u},y_{i})}{\partial x_{u}}&=\sum\limits_{i\in I_{u}}(-2y_{i})c_{ui}(p_{ui}-x_{u}^{T}y_{i})+2\lambda x_{u}
\\&=-2Y^{T}C^{u}(P_{u}-(x_{u}^{T}Y^{T})^{T})+\lambda x_{u}
\\&=-2Y^{T}C^{u}(P_{u}-Yx_{u})+\lambda x_{u}
\end{align}

process 3 : Let the partial derivation equal to 0, and get

\begin{align}
Y^{T}C^{u}P_{u}&=Y^{T}C^{u}Yx_{u}+\lambda x_{u}
\\x_{u}&=(Y^{T}C^{u}Y+\lambda I)^{-1}Y^{T}C^{u}P_{u}
\end{align}

Similarly available : ( Fix $X$ )
\begin{align}
y_{i}=(X^{T}C^{i}X+\lambda I)^{-1}X^{T}C^{i}P_{i}
\end{align}

A computational bottleneck here is computing $Y^{T}C^{u}Y$ , whose naive calculation will require time $O(f^{2}n)$ (for each of the $m$ users). A significant speedup is achieved by using the fact that :

<center>$Y^{T}C^{u}Y=Y^{T}Y+Y^{T}(C^{u}-I)Y$

Now,$Y^{T}Y$ is independent of $u$ and was **already precomputed**.You don't have to compute this number for each of the $m$ users( compute $m$ times ). what's more, $C^{u}-I$ **has only $n_{u}$ non-zero elements**, where $n_{u}$ is the number of items for which $r_{ui} > 0$ and typically $n_{u} ≪ n$. Consequently, recomputation of $x_{u}$ is performed in time $O(f^{2}n_{u} + f^{3})$.

Then we solve ：

\begin{align}
x_{u}&=(Y^{T}Y+Y^{T}(C^{u}-I)Y+\lambda I)^{-1}Y^{T}C^{u}P_{u}\tag{5}
\\y_{i}&=(X^{T}X+X^{T}(C^{i}-I)X+\lambda I)^{-1}X^{T}C^{i}P_{i}\tag{6}
\end{align}